In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
using Printf
using JSON
using CSV
using DataFrames
using LaTeXStrings
using BenchmarkTools
LaTeXStrings

LaTeXStrings

In [12]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module
param3b = SailorMoon.dynamics_parameters()
paramMulti = SailorMoon.multi_shoot_parameters(param3b)
plotly()
fs = 16

┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs\Docs.jl:240


16

## obtain an initial guess

In [13]:
### PARAMETERS ###################################
# csv file to load the initial solution
filename = "../run/data/grid_search_Tsit5_0525_EMrotThrust.csv"
# dv_dir function corresponding to the csv file 
dir_func = SailorMoon.dv_EMrotdir_sb1frame 

# parameters
param3b = SailorMoon.dynamics_parameters()
paramMulti = SailorMoon.multi_shoot_parameters(param3b)

n_arc = 5
##################################################

if dir_func == SailorMoon.dv_no_thrust
    τ_ig = 0.0
else 
    τ_ig = 1.0
end

# load initial guess
df = DataFrame(CSV.File(filename))

# maybe want to use "for row in eachrow(df)" to automate the process...? 
row = df[21,:]


Row,id,phi0,epsr,epsv,thetasf,rp_kep,ra_kep,alpha,ra,dt1,dt2,x_lpo,y_lpo,z_lpo,xdot_lpo,ydot_lpo,zdot_lpo,m_lpo,x_ra,y_ra,z_ra,xdot_ra,ydot_ra,zdot_ra,m_ra,x_rp,y_rp,z_rp,xdot_rp,ydot_rp,zdot_rp,m_rp,x_lr,y_lr,z_lr,xdot_lr,ydot_lr,zdot_lr,m_lr,t_lr,tof,lfb
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
21,21.0,0.397935,1.0e-5,1.0e-5,5.44543,0.00808627,2.2964,-2.60377,2.84033,9.48719,6.08247,388.073,-0.902908,-1.62381e-21,0.717073,-0.644585,-1.49526e-21,1.0,391.481,0.995379,-4.82231e-21,-0.0136708,0.0414753,-5.25646e-22,1.01762,388.821,-0.0143727,0.0,7.99321,-13.4007,0.0,1.03253,388.821,-0.0143727,1.15655e-23,7.99321,-13.4007,2.66139e-19,1.03253,15.5697,15.5697,0.0


In [14]:
x0, lx, ux = SailorMoon.make_ig_bounds2(row, τ_ig, paramMulti.n_arc)
fitness!, ng, lg, ug, eval_sft = SailorMoon.get_fitness2(dir_func, paramMulti, x0);

In [15]:
xvar = vcat(x0[1:2], x0[4:5])
@time DF = SailorMoon.get_jacobian2(eval_sft, x0, xvar, ng)

  8.774542 seconds (24.93 M allocations: 1.360 GiB, 4.00% gc time, 8.68% compilation time)


16×4 Matrix{Float64}:
 43978.8          70345.4          70318.4          70363.9
  6013.49          9287.74          9283.87          9289.8
     7.34631e-17      1.28605e-16      1.28561e-16      1.28644e-16
 23654.9          37785.4          37770.8          37795.2
  2195.48          3473.8           3472.43          3474.66
     6.33315e-18      1.11557e-17      1.11519e-17      1.11592e-17
     5.55112e-9      -1.55431e-8      -8.88178e-9       4.44089e-9
     0.0              0.0              0.0              0.0
     0.0              0.0              0.0              0.0
     0.0              0.0              0.0              0.0
     0.0              0.0              0.0              0.0
     0.0              0.0              0.0              0.0
     0.0              0.0              0.0              0.0
     0.0              0.0              0.0              0.0
    -0.858825        -1.37109         -1.37109         -1.37109
     7.99321         -5.40749         -5.41444    

In [16]:
@time inv(DF*transpose(DF))

LoadError: SingularException(8)

In [7]:
@time x_iter = SailorMoon.newton_update(xvar, eval_sft(x0), DF)

LoadError: LAPACKException(1)

In [17]:
transpose(DF)*DF

4×4 Matrix{Float64}:
 2.53468e9  4.051e9    4.04944e9  4.05206e9
 4.051e9    6.47455e9  6.47205e9  6.47624e9
 4.04944e9  6.47205e9  6.46955e9  6.47374e9
 4.05206e9  6.47624e9  6.47374e9  6.47793e9

In [ ]:
xs, fs, converged = SailorMoon.differential_correction2(x0, eval_sft)

iter #